In [ ]:
from dotenv import load_dotenv
from typing import Optional
import os
import yaml
import pandas as pd
import vectorbt as vbt
import numpy as np

load_dotenv()
config_path = os.getenv('3CANDLES_CONFIG_PATH')

with open(config_path, 'r') as file:
    config = yaml.safe_load(file)
    
df_hour = pd.read_csv(config['Data_filename_hour'])
df_hour = df_hour.set_index('Time')
df_hour.index = pd.to_datetime(df_hour.index)

df_hour = df_hour.drop(columns=['Volume', 'High', 'Low'])
df_hour['Dir'] = 0

df_hour.loc[df_hour['Close'] > df_hour['Open'], 'Dir'] = 1
bull_entry_mask = ((df_hour['Dir'] == 1) & (df_hour['Dir'].shift(1) == 1) & (df_hour['Dir'].shift(2) == 1) & #short
                    (df_hour['Close'].shift(2) < df_hour['Open']))
df_hour['Bull Entry'] = bull_entry_mask
bear_entry_mask = ((df_hour['Dir'] == 0) & (df_hour['Dir'].shift(1) == 0) & (df_hour['Dir'].shift(2) == 0) & #long
                    (df_hour['Close'].shift(2) > df_hour['Open']))
df_hour['Bear Entry'] = bear_entry_mask

df_hour.loc[df_hour['Bull Entry'] == True, 'SL'] = df_hour['Close'] + ((df_hour['Close'] - df_hour['Close'].shift(2)) * (config['RR'] * 0.5)) #short
df_hour.loc[df_hour['Bear Entry'] == True, 'SL'] = df_hour['Close'] - ((df_hour['Close'].shift(2) - df_hour['Close']) * (config['RR'] * 0.5))#long

df_hour.loc[df_hour['Bull Entry'] == True, 'TP'] = df_hour['Close'] - ((df_hour['Close'] - df_hour['Close'].shift(2)) * config['RR']) #short
df_hour.loc[df_hour['Bear Entry'] == True, 'TP'] = df_hour['Close'] + ((df_hour['Close'].shift(2) - df_hour['Close']) * config['RR']) #long

index_arr_hour = df_hour.index.to_numpy()
open_arr_hour = df_hour['Open'].to_numpy()
close_arr_hour = df_hour['Close'].to_numpy()
bull_entry_arr_hour = df_hour['Bull Entry'].to_numpy()
bear_entry_arr_hour = df_hour['Bear Entry'].to_numpy()
sl_arr_hour = df_hour['SL'].to_numpy()
tp_arr_hour = df_hour['TP'].to_numpy()
price_arr_hour = np.full(len(index_arr_hour), np.nan)
bull_entrymask_arr_hour = np.full(len(index_arr_hour), False)
bear_entrymask_arr_hour = np.full(len(index_arr_hour), False)
bull_exit_arr_hour = np.full(len(index_arr_hour), False)
bear_exit_arr_hour = np.full(len(index_arr_hour), False)

trade_direct: Optional[str] = None
cur_sl: Optional[float] = None; cur_tp: Optional[float] = None

for i in range(len(index_arr_hour)):
    if trade_direct is None:

        if bull_entry_arr_hour[i]:
            trade_direct = 'bull'
            bull_entrymask_arr_hour[i] = True
            cur_sl, cur_tp = sl_arr_hour[i], tp_arr_hour[i]
            price_arr_hour[i] = close_arr_hour[i]
        elif bear_entry_arr_hour[i]:
            trade_direct = 'bear'
            bear_entrymask_arr_hour[i] = True
            cur_sl, cur_tp = sl_arr_hour[i], tp_arr_hour[i]
            price_arr_hour[i] = close_arr_hour[i]
        else:
            continue  

    elif trade_direct == 'bull':

        if close_arr_hour[i] >= cur_sl:
            price_arr_hour[i] = cur_sl
            trade_direct, cur_sl, cur_tp = None, None, None
            bull_exit_arr_hour[i] = True
        elif close_arr_hour[i] <= cur_tp:
            price_arr_hour[i] = cur_tp
            trade_direct, cur_sl, cur_tp = None, None, None
            bull_exit_arr_hour[i] = True

    elif trade_direct == 'bear':

        if close_arr_hour[i] <= cur_sl:
            price_arr_hour[i] = cur_sl
            trade_direct, cur_sl, cur_tp = None, None, None
            bear_exit_arr_hour[i] = True
        elif close_arr_hour[i] >= cur_tp:
            price_arr_hour[i] = cur_tp
            trade_direct, cur_sl, cur_tp = None, None, None
            bear_exit_arr_hour[i] = True

pf = vbt.Portfolio.from_signals(
entries = bear_entrymask_arr_hour,
exits = bear_exit_arr_hour,
short_entries = bull_entrymask_arr_hour,
short_exits = bull_exit_arr_hour,
price = price_arr_hour,
open = df_hour["Open"],
close = df_hour["Close"],
size = config['Trade']['size'],
size_type = config['Trade']['size_type'],
fees = config['Broker']['fees'],
fixed_fees = config['Broker']['fixed_fees'],
slippage = config['Slippage'],
init_cash = config['Initial_cash'],
freq = config['Frequency']
)

ValueError: operands could not be broadcast together with shapes (12048,) (2,) 

In [ ]:
pf.stats()

Start                         2024-01-01 22:00:00+00:00
End                           2025-12-05 21:00:00+00:00
Period                                502 days 00:00:00
Start Value                                     50000.0
End Value                                  58254.388858
Total Return [%]                              16.508778
Benchmark Return [%]                           5.421458
Max Gross Exposure [%]                        30.213841
Total Fees Paid                                     0.0
Max Drawdown [%]                               0.403712
Max Drawdown Duration                  17 days 22:00:00
Total Trades                                       1362
Total Closed Trades                                1361
Total Open Trades                                     1
Open Trade PnL                                 14.26681
Win Rate [%]                                  20.646583
Best Trade [%]                                 0.768921
Worst Trade [%]                                 

In [ ]:
pf.plot().show()

In [ ]:
pf.trades.records_readable.tail(21)

,Exit Trade Id,Column,Size,Entry Timestamp,Avg Entry Price,Entry Fees,Exit Timestamp,Avg Exit Price,Exit Fees,PnL,Return,Direction,Status,Position Id
1341,1341,0,15038.530433,2025-11-28 04:00:00+00:00,1.15895,0.0,2025-11-28 05:00:00+00:00,1.15895,0.0,0.000000,0.000000,Short,Closed,1341
1342,1342,0,15063.355497,2025-11-28 08:00:00+00:00,1.15704,0.0,2025-11-28 09:00:00+00:00,1.15704,0.0,0.000000,0.000000,Long,Closed,1342
1343,1343,0,15077.950762,2025-11-28 10:00:00+00:00,1.15592,0.0,2025-11-28 14:00:00+00:00,1.15816,0.0,33.774610,0.001938,Long,Closed,1343
1344,1344,0,15024.068463,2025-11-28 15:00:00+00:00,1.16074,0.0,2025-12-01 08:00:00+00:00,1.16074,0.0,0.000000,0.000000,Short,Closed,1344
1345,1345,0,15004.936439,2025-12-01 09:00:00+00:00,1.16222,0.0,2025-12-01 10:00:00+00:00,1.16222,0.0,0.000000,0.000000,Short,Closed,1345
1346,1346,0,14997.709995,2025-12-01 11:00:00+00:00,1.16278,0.0,2025-12-01 12:00:00+00:00,1.16278,0.0,0.000000,0.000000,Short,Closed,1346
1347,1347,0,14996.162377,2025-12-01 15:00:00+00:00,1.16290,0.0,2025-12-01 16:00:00+00:00,1.16290,0.0,0.000000,0.000000,Long,Closed,1347
1348,1348,0,15006.873276,2025-12-01 17:00:00+00:00,1.16207,0.0,2025-12-01 18:00:00+00:00,1.16207,0.0,0.000000,0.000000,Long,Closed,1348
1349,1349,0,15022.644810,2025-12-01 19:00:00+00:00,1.16085,0.0,2025-12-01 23:00:00+00:00,1.16085,0.0,0.000000,0.000000,Long,Closed,1349
1350,1350,0,15026.139713,2025-12-02 00:00:00+00:00,1.16058,0.0,2025-12-02 03:00:00+00:00,1.16120,0.0,9.316207,0.000534,Long,Closed,1350
